<a href="https://colab.research.google.com/github/piyush-bhandari-commits/self-learn-repo/blob/master/Text_Classification_AutoML_and_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install default-jre
!java -version

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
openjdk version "11.0.8" 2020-07-14
OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)


In [2]:
!pip install h2o

In [80]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
import h2o
from h2o.automl import H2OAutoML
import seaborn as sns
from pprint import pprint

In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.8" 2020-07-14; OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1); OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpm1qptwbm
  JVM stdout: /tmp/tmpm1qptwbm/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpm1qptwbm/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.1
H2O_cluster_version_age:,16 days
H2O_cluster_name:,H2O_from_python_unknownUser_82gi41
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [5]:
# Fetching the dataset from the data.gov website
url = 'https://files.consumerfinance.gov/ccdb/complaints.csv.zip'
actual_data = pd.read_csv(url, compression='zip')

In [6]:
# Pre-preocessing the actual dataset
print ('Shape of Actual Data: {}'.format(actual_data.shape))

Shape of Actual Data: (1727583, 18)


In [7]:
actual_data.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2019-09-24,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,transworld systems inc. \nis trying to collect...,NaN,TRANSWORLD SYSTEMS INC,FL,335XX,NaN,Consent provided,Web,2019-09-24,Closed with explanation,Yes,NaN,3384392
1,2019-09-19,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,PA,15206,NaN,Consent not provided,Web,2019-09-20,Closed with non-monetary relief,Yes,NaN,3379500
2,2019-10-25,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,I would like to request the suppression of the...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",CA,937XX,NaN,Consent provided,Web,2019-10-25,Closed with explanation,Yes,NaN,3417821
3,2019-11-08,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,"Over the past 2 weeks, I have been receiving e...",NaN,"Diversified Consultants, Inc.",NC,275XX,NaN,Consent provided,Web,2019-11-08,Closed with explanation,Yes,NaN,3433198
4,2019-02-08,Vehicle loan or lease,Lease,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,NaN,NaN,HYUNDAI CAPITAL AMERICA,FL,33161,NaN,Consent not provided,Web,2019-02-08,Closed with non-monetary relief,Yes,NaN,3146310


In [8]:
raw_dataset = actual_data[[actual_data.columns[1], actual_data.columns[5], actual_data.columns[7]]].rename(columns={'Consumer complaint narrative': 'Complaint'})
clean_raw_dataset = raw_dataset.dropna()
print('The dataset has {} rows and {} columns'.format(clean_raw_dataset.shape[0], clean_raw_dataset.shape[1]))
clean_raw_dataset.head()

The dataset has 585178 rows and 3 columns


,Product,Complaint,Company
0,Debt collection,transworld systems inc. \nis trying to collect...,TRANSWORLD SYSTEMS INC
2,"Credit reporting, credit repair services, or o...",I would like to request the suppression of the...,"TRANSUNION INTERMEDIATE HOLDINGS, INC."
3,Debt collection,"Over the past 2 weeks, I have been receiving e...","Diversified Consultants, Inc."
8,"Money transfer, virtual currency, or money ser...","I was sold access to an event digitally, of wh...","Paypal Holdings, Inc"
9,Mortgage,This complaint dates back to last XX/XX/XXXX d...,NATIONSTAR MORTGAGE


In [9]:
clean_raw_dataset['Product'].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    197065
Debt collection                                                                 120326
Mortgage                                                                         68025
Credit card or prepaid card                                                      41344
Credit reporting                                                                 31588
Student loan                                                                     26750
Checking or savings account                                                      23911
Credit card                                                                      18838
Bank account or service                                                          14885
Money transfer, virtual currency, or money service                               10134
Vehicle loan or lease                                                            10047
Consumer Loan                              

In [10]:
dataset = clean_raw_dataset.sample(n=100000)
dataset['Product'].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    33798
Debt collection                                                                 20379
Mortgage                                                                        11600
Credit card or prepaid card                                                      6935
Credit reporting                                                                 5349
Student loan                                                                     4659
Checking or savings account                                                      4072
Credit card                                                                      3252
Bank account or service                                                          2598
Money transfer, virtual currency, or money service                               1749
Vehicle loan or lease                                                            1717
Consumer Loan                                         

In [44]:
label_count = dataset['Product'].value_counts().to_dict()
for label, count in label_count.items():
    if count < 3000:
        dataset = dataset.drop(dataset[dataset.Product.values == label].index)

In [45]:
dataset.Product.value_counts()

Credit reporting, credit repair services, or other personal consumer reports    33798
Debt collection                                                                 20379
Mortgage                                                                        11600
Credit card or prepaid card                                                      6935
Credit reporting                                                                 5349
Student loan                                                                     4659
Checking or savings account                                                      4072
Credit card                                                                      3252
Name: Product, dtype: int64

In [46]:
dataset.Company.value_counts()

EQUIFAX, INC.                             11314
Experian Information Solutions Inc.       10292
TRANSUNION INTERMEDIATE HOLDINGS, INC.    10221
CITIBANK, N.A.                             2951
Navient Solutions, LLC.                    2562
                                          ...  
FIRSTRUST MORTGAGE, INC.                      1
AAA Collections                               1
PARKSIDE LENDING, LLC                         1
Lima One Capital, LLC                         1
D.B.F. COLLECTION CORP.                       1
Name: Company, Length: 2589, dtype: int64

In [61]:
mapping = {}
for value in np.arange(len(np.unique(dataset.Product.values))):
    mapping[np.unique(dataset.Product.values)[value]] = value 

In [62]:
mapping

{'Checking or savings account': 0,
 'Credit card': 1,
 'Credit card or prepaid card': 2,
 'Credit reporting': 3,
 'Credit reporting, credit repair services, or other personal consumer reports': 4,
 'Debt collection': 5,
 'Mortgage': 6,
 'Student loan': 7}

In [67]:
dataset['Target'] = dataset.Product.map(mapping)
dataset.head()

,Product,Complaint,Company,Target
1718709,"Credit reporting, credit repair services, or o...",Chapter XXXX bankruptcy is invalidly reporting...,"EQUIFAX, INC.",4
1580135,Debt collection,"Company FBCS , inc XXXX XXXX XXXX XXXX XXXX. X...",Financial Business and Consumer Solutions.,5
273732,Mortgage,I had a 30-year mortgage with Pentagon Federal...,PENTAGON FEDERAL CREDIT UNION,6
105995,"Credit reporting, credit repair services, or o...",Please help me. I am asking for you to look in...,"TIDEWATER FINANCE COMPANY, INC.",4
1688241,"Credit reporting, credit repair services, or o...",Since I am a veteran and it involves the FCRA ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",4


In [68]:
# Splitting data into train and test set
train, test = train_test_split(dataset, test_size=0.20, random_state=42)

In [72]:
print ('Shape of Train Data : {}'.format(train.shape))
print ('Shape of Test Data : {}'.format(test.shape))

Shape of Train Data : (72035, 4)
Shape of Test Data : (18009, 4)


In [83]:
pprint (train['Complaint'].values[10])

('Midland Funding has continued to report inconsistent rention dates on '
 'multiple accounts, to the XXXX credit agencies. On account XXXX, they report '
 'to XXXX a retention date of XX/XX/XXXX -- XXXX XX/XX/XXXX -- no reporting on '
 'XXXX. Account XXXX they report to XXXX a rention date of XX/XX/XXXX -- no '
 'reporting to XXXX -- and a reporting of XX/XX/XXXX to XXXX. On account XXXX '
 'they do report to XXXX or XXXX -- to XXXX they report a rention date of '
 'XX/XX/XXXX. On account XXXX they do not report to XXXX & XXXX -- but report '
 'a rention date of XX/XX/XXXX. On these accounts, they do not report an '
 'original creditor or contact information for the original creditor. They '
 'have failed to report correct information on any account that they hold of '
 'mine. I have submitted numerous disputes with each credit agency, & they '
 'continue to verify each account with each agency, even though all '
 'information that they report is inconsistent.')


In [87]:
pprint (train['Complaint'].values[28])

('I have an old XXXX XXXX account from Household Finance ( Account # XXXX ). '
 'The asset acceptance tactics for recovering date have caused hardship on my '
 'family as well as third parties ( relatives living outside my home ). XXXX '
 'XXXX has robo called and intimidated my family by telling them if I do not '
 'pay the debt I will go to Jail ... .needless to say my elderly mother was '
 'terrified for my well being. XXXX XXXX has violated my rights as a consumer '
 'and deployed terrorizing tactics to my family and myself. These tactics are '
 "listed in the FTC 's XXXX counter compliant charged against XXXX XXXX : 1. "
 'repeatedly calling third parties who do not owe a debt ; 2. informing third '
 'parties about a debt ; 3. using illegal debt-collection practices, including '
 'misrepresenting the character, amount, or legal status of a debt ; providing '
 'inaccurate information to credit reporting agencies ; and making false '
 'representations to collect a debt')
